### 前言

前面我们已经介绍了通过梯度下降法来求解逻辑回归的闭式解的形式:
$$\beta \leftarrow \beta-\gamma \triangledown J(\beta)$$
$$\triangledown J(\beta)=(y-h_{\beta}(x))*x$$
其中$h_{\beta}(x)$为:
$$h_{\beta}(x)=\frac{e^{\beta^T\hat{x}}}{1+e^{\beta^T\hat{x}}}$$

**逻辑回归的本质是线性分类，也就是通过一条直线将样本划分为两类。**

### 从零实现逻辑回归

In [1]:
import numpy as np
import pandas as pd

In [2]:
#定义逻辑回归模型
class LogisticRegression():
    def __init__(self,lr=0.0001,epoch=500):
        self.w=None
        self.lr=lr  #学习率
        self.epoch=epoch   #迭代次数
        
    def sigmoid(self,X):
        return 1/(1.0+np.exp(-X))
    
    def fit(self,X,Y):
        m,n=X.shape
        self.w=np.ones((n,1))
        Y=Y.reshape((-1,1))#转为列向量
        for k in range(self.epoch):
            h=self.sigmoid(X.dot(self.w))
            error=(Y-h)
            self.w=self.w-self.lr*(X.T.dot(error))
    
    def predict(self,X):
        y_pred=self.sigmoid(X.dot(self.w))
        return np.squeeze(np.round(y_pred).reshape((1,-1)))  #对结果四舍五入

In [3]:
#读取数据
dataset=pd.read_csv("datasets/Social_Network_Ads.csv")
X=dataset.iloc[:,[2,3]].values
Y=dataset.iloc[:,4].values

In [4]:
#划分训练集和测试集
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=0)

In [5]:
#标准化数据
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [6]:
#训练模型
model=LogisticRegression()
model.fit(X_train,Y_train)

In [7]:
#预测
y_pred=model.predict(X_test)
print(y_pred)

[1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0.
 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0.
 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1.
 1. 0. 1. 1. 0. 0. 0. 0.]


In [8]:
#可视化
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(Y_test,y_pred)
print(cm)

[[13 45]
 [18  4]]


### 使用随机梯度下降法来优化

从上述结果可以看出，使用随机梯度下降法在数据量较多时，复杂度较大，所以我们通过随机梯度下降法来优化。

随机梯度下降法即**每次仅用一个样本来回归系数。**